In [6]:
%pip install pandas tqdm requests langchain openai datasets ragas
%pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import tqdm
import requests
from datasets import Dataset
import os
import getpass

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

/home/svyatoslav/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/svyatoslav/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/svyatoslav/.local/lib/python3.10/site-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_cor

In [4]:
os.environ["OPENAI_API_KEY"] = ""

In [5]:
import random

# clear ml credentials
dataset = "v2_ragas_npa_dataset_firstPart.xlsx"

questions_df = pd.read_excel(open(dataset, "rb"))
questions_df.drop(index=[0], inplace=True)
# questions = questions_df["question"].to_list()

FileNotFoundError: [Errno 2] No such file or directory: 'v2_ragas_npa_dataset_firstPart.xlsx'

In [26]:
def get_system_responses(questions):
    """
    Send requests to the /ask endpoint and collect responses.
    """
    for question in tqdm.tqdm(questions.head(100).itertuples()):
        try:
            response = requests.post("http://localhost:8000/rag/query", json={"query": question[1]})
            response.raise_for_status()
            response_data = response.json()
            print(response_data)
            yield {
                    "question": question[1],
                    "response": response_data["answer"],
                    "ground_truth": question[3],
                    "context": response_data["contexts"], 
                }
        except Exception as e:
            print(f"Error while sending query: {question[0]}, error message: {e}")
            continue

In [ ]:
system_responses = [items for items in get_system_responses(questions_df)]
system_responses

In [ ]:
data_samples = {
    "question": [response["question"] for response in system_responses],
    "answer": [response["response"] for response in system_responses],
    "contexts": [response["context"] if type(response["context"]) == list else [response["context"]] for response in system_responses],
    "ground_truth": [response["ground_truth"] for response in system_responses],
}

dataframe = pd.DataFrame().from_dict(data_samples)
dataframe.drop(index=[1], inplace=True)

dataset = Dataset.from_dict(dataframe)

In [ ]:
dataframe

In [ ]:
result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

In [40]:
# {'context_precision': 0.8770, 'faithfulness': 0.5064, 'answer_relevancy': 0.4106, 'context_recall': 0.9133}
# {'context_precision': 1.0000, 'faithfulness': 0.7475, 'answer_relevancy': 0.7865, 'context_recall': 0.8148}